# [Demo] Robust Node Classification on Graphs: Jointly from Bayesian Label Transition and Topology-based Label Propagation (LInDT)

### Authors: Jun Zhuang, Mohammad Al Hasan
```
The paper is accepted for CIKM 2022
```

In [1]:
import os
import sys
import time
import argparse
from datetime import datetime
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import dgl
from sklearn.metrics import accuracy_score, f1_score
from scipy.stats import entropy
from load_data import LoadDataset
from models_GNN import GNN
from models_LInDT import LInDT
from train_GNN import train, evaluation, prediction
from rdmPert import random_connections
from utils import read_yaml_file, read_pickle, dump_pickle, split_masks, generate_random_noise_label, \
                    load_checkpoint, save_checkpoint, count_parameters, \
                    compute_accuracy, compute_f1_score, dist_pre_class, compute_entropy, \
                    compute_edge_homo_ratio, select_target_nodes, \
                    gen_init_trans_matrix


In [2]:
# Read config files for training
config_file1 = read_yaml_file("../config", "model_lindt")
assert config_file1
train_gnn_config = config_file1["train_gnn"]
data_name = train_gnn_config["data_name"]
model_name = train_gnn_config["model_name"]
NOISE_RATIO = train_gnn_config["NOISE_RATIO"]
NUM_EPOCHS = train_gnn_config["NUM_EPOCHS"]
is_trainable = train_gnn_config["is_trainable"]
GPU = train_gnn_config["GPU"]

# Read config files for perturbations
config_file2 = read_yaml_file("../config", "perturbations")
assert config_file2
pert_config = config_file2["rdmPert"]
TARGET_CLASS = pert_config["TARGET_CLASS"]
SAMPLE_RATE = pert_config["SAMPLE_RATE"]

# Read config files for inference
infer_config = config_file1["inference"]
pert_type = infer_config["pert_type"]
attack_type = infer_config["attack_type"]
is_inferable = infer_config["is_inferable"]
infer_type = infer_config["infer_type"]
topo_type = infer_config["topo_type"]
NUM_EPS = infer_config["NUM_EPS"]
NUM_RETRAIN = infer_config["NUM_RETRAIN"]
WARMUP_STEP = infer_config["WARMUP_STEP"]
is_alert = infer_config["is_alert"]
is_dynamic_phi = infer_config["is_dynamic_phi"]
is_dynamic_alpha = infer_config["is_dynamic_alpha"]
ALPHA = infer_config["ALPHA"]

# Initialize the parameters
CUT_RATE = [0.1, 0.2]  # the split ratio of train/validation mask.
LR = 0.001
N_LAYERS = 2
N_HIDDEN = 200
DROPOUT = 0
WEIGHT_DECAY = 0
kwargs_dicts = {"GCN": [None, None, None],  # aggregator_type, n_filter, n_heads.
                "GraphSAGE": ["gcn", None, None],
                "SGC": [None, 2, None],
                "GAT": [None, None, 3]}
topo_dicts = {"random": "random_kernel",
               "major": "majority_kernel",
               "degree": "degree_weighted_kernel"}
Kernel_Type = topo_dicts[topo_type] if topo_type in topo_dicts.keys() else ""
PERT_RATE = 0.01  # num_perturbators cannot exceed this ratio
assert SAMPLE_RATE + PERT_RATE <= 1.0
dirs_attack = '../data/attacker_data/'


## Train the node classifier

In [3]:
# Load dataset
data = LoadDataset(data_name)
graph = data.load_data()
feat, label = graph.ndata['feat'], graph.ndata['label']
print("Class ID: ", set(label.numpy()))
# Randomly split the train, validation, test mask by given cut rate
train_mask, val_mask, test_mask = split_masks(label, cut_rate=CUT_RATE)
# Generate noisy label
Y_noisy = generate_random_noise_label(label, noisy_ratio=NOISE_RATIO, seed=0)
Y_noisy = torch.LongTensor(Y_noisy)
dump_pickle('../data/noisy_label/Y_gt_noisy_masks.pkl', \
            [label, Y_noisy, train_mask, val_mask, test_mask])
# Display the variables
print("""-------Data statistics-------'
      # Nodes: {0}
      # Edges: {1}
      # Features: {2}
      # Classes: {3}
      # Train samples: {4}
      # Val samples: {5}
      # Test samples: {6}
      # The edge homophily ratio: {7}%
      """.format(graph.number_of_nodes(), graph.number_of_edges(),\
                 feat.shape[1], len(torch.unique(label)), \
                  train_mask.int().sum().item(), \
                  val_mask.int().sum().item(), \
                  test_mask.int().sum().item(), \
                  np.round(compute_edge_homo_ratio(graph, label)*100, 2)
                  ))

# Setup the gpu if necessary
if GPU < 0:
    print("Using CPU!")
    cuda = False
else:
    print("Using GPU!")
    cuda = True
    torch.cuda.set_device(GPU)
    graph = graph.to('cuda')
    feat = feat.cuda()
    label = label.cuda()
    Y_noisy = Y_noisy.cuda()
    train_mask = train_mask.cuda()
    val_mask = val_mask.cuda()
    test_mask = test_mask.cuda()

# Initialize the node classifier
model = GNN(g=graph,
            in_feats=feat.shape[1],
            n_hidden=N_HIDDEN,
            n_classes=len(torch.unique(label)),
            n_layers=N_LAYERS,
            activation=F.relu,
            dropout=DROPOUT,
            model_name=model_name,
            aggregator_type=kwargs_dicts[model_name][0],
            n_filter=kwargs_dicts[model_name][1],
            n_heads=kwargs_dicts[model_name][2])
print(model)
print(f'The model has {count_parameters(model):,} trainable parameters.')
if cuda:  # if gpu is available
    model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
# Path for saving the parameters
dirs = 'runs/{0}_{1}/'.format(data_name, model_name)
path = dirs + 'model_best.pth.tar'

# Training the model
if is_trainable:
    train(model, optimizer, dirs, feat, Y_noisy, train_mask, val_mask, NUM_EPOCHS)
    # Save the weights of current model
    W0, W1 = model.w0.cpu(), model.w1.cpu()
    dump_pickle('../data/attacker_data/W_{0}_{1}.pkl'\
                .format(data_name, model_name),
                [W0.detach().numpy(), W1.detach().numpy()])

# Evaluation
print("Evaluation on stationary graphs!")
evaluation(model, optimizer, path, graph, feat, label, test_mask, cuda)
# Generate and save predicted labels
Y_pred, Y_pred_sm = prediction(model, optimizer, path, graph, feat)  # prediction on all nodes
if cuda:
    Y_pred, Y_pred_sm = Y_pred.cpu(), Y_pred_sm.cpu()
dump_pickle('../data/noisy_label/Y_preds.pkl', [Y_pred, Y_pred_sm])
print("Y_pred/Y_pred_sm.shape: ", Y_pred.shape, Y_pred_sm.shape)


Current dataset: cora, citeseer, pubmed, amazoncobuy, coauthor, reddit.
Selecting cora Dataset ...
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Data is stored in: /Users/[user_name]/.dgl
cora Dataset Loaded!
Class ID:  {0, 1, 2, 3, 4, 5, 6}
-------Data statistics-------'
      # Nodes: 2708
      # Edges: 13264
      # Features: 1433
      # Classes: 7
      # Train samples: 270
      # Val samples: 542
      # Test samples: 1896
      # The edge homophily ratio: 81.0%
      
Using CPU!
GNN(
  (layers): ModuleList(
    (0): GraphConv(in=1433, out=200, normalization=both, activation=<function relu at 0x7fb48f9644c0>)
    (1): Dropout(p=0, inplace=False)
    (2): GraphConv(in=200, out=200, normalization=both, activation=<function relu at 0x7fb48f9644c0>)
    (3): Dropout(p=0, inplace=False)
    (4): GraphConv(in=200, out=7, normalization=both, activatio

/Users/junzhuang/opt/miniconda3/envs/py39/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/Users/junzhuang/opt/miniconda3/envs/py39/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/junzhuang/opt/miniconda3/envs/py39/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 00002 | Time(s) nan | Train Loss 1.9401 | Val Loss 1.9413 | Train Accuracy 0.3074 | Val Accuracy 0.2915 
Epoch 00003 | Time(s) 0.1048 | Train Loss 1.9349 | Val Loss 1.9369 | Train Accuracy 0.2963 | Val Accuracy 0.2934 
Epoch 00004 | Time(s) 0.0873 | Train Loss 1.9292 | Val Loss 1.9322 | Train Accuracy 0.2963 | Val Accuracy 0.2952 
Epoch 00005 | Time(s) 0.0806 | Train Loss 1.9227 | Val Loss 1.9268 | Train Accuracy 0.2926 | Val Accuracy 0.2952 
Epoch 00006 | Time(s) 0.0770 | Train Loss 1.9155 | Val Loss 1.9208 | Train Accuracy 0.2889 | Val Accuracy 0.2934 
Epoch 00007 | Time(s) 0.0741 | Train Loss 1.9074 | Val Loss 1.9142 | Train Accuracy 0.2889 | Val Accuracy 0.2934 
Epoch 00008 | Time(s) 0.0729 | Train Loss 1.8985 | Val Loss 1.9068 | Train Accuracy 0.2889 | Val Accuracy 0.2934 
Epoch 00009 | Time(s) 0.0719 | Train Loss 1.8886 | Val Loss 1.8988 | Train Accuracy 0.2889 | Val Accuracy 0.2934 
Epoch 00010 | Time(s) 0.0713 | Train Loss 1.8779 | Val Loss 1.8902 | Train Accuracy 0.2889 

Epoch 00076 | Time(s) 0.0655 | Train Loss 0.5515 | Val Loss 1.0286 | Train Accuracy 0.8741 | Val Accuracy 0.7103 
Epoch 00077 | Time(s) 0.0654 | Train Loss 0.5353 | Val Loss 1.0231 | Train Accuracy 0.8778 | Val Accuracy 0.7085 
Epoch 00078 | Time(s) 0.0654 | Train Loss 0.5196 | Val Loss 1.0182 | Train Accuracy 0.8778 | Val Accuracy 0.7085 
Epoch 00079 | Time(s) 0.0654 | Train Loss 0.5044 | Val Loss 1.0139 | Train Accuracy 0.8852 | Val Accuracy 0.7103 
Epoch 00080 | Time(s) 0.0653 | Train Loss 0.4895 | Val Loss 1.0100 | Train Accuracy 0.8889 | Val Accuracy 0.7103 
Epoch 00081 | Time(s) 0.0653 | Train Loss 0.4752 | Val Loss 1.0067 | Train Accuracy 0.8889 | Val Accuracy 0.7085 
Epoch 00082 | Time(s) 0.0653 | Train Loss 0.4612 | Val Loss 1.0039 | Train Accuracy 0.9037 | Val Accuracy 0.7122 
Epoch 00083 | Time(s) 0.0653 | Train Loss 0.4477 | Val Loss 1.0015 | Train Accuracy 0.9037 | Val Accuracy 0.7122 
Epoch 00084 | Time(s) 0.0653 | Train Loss 0.4346 | Val Loss 0.9996 | Train Accuracy 0.90

Epoch 00148 | Time(s) 0.0651 | Train Loss 0.0794 | Val Loss 1.2372 | Train Accuracy 0.9852 | Val Accuracy 0.7325 
Epoch 00149 | Time(s) 0.0651 | Train Loss 0.0777 | Val Loss 1.2421 | Train Accuracy 0.9889 | Val Accuracy 0.7325 
Epoch 00150 | Time(s) 0.0651 | Train Loss 0.0761 | Val Loss 1.2469 | Train Accuracy 0.9889 | Val Accuracy 0.7325 
Epoch 00151 | Time(s) 0.0651 | Train Loss 0.0744 | Val Loss 1.2517 | Train Accuracy 0.9889 | Val Accuracy 0.7325 
Epoch 00152 | Time(s) 0.0651 | Train Loss 0.0729 | Val Loss 1.2565 | Train Accuracy 0.9889 | Val Accuracy 0.7306 
Epoch 00153 | Time(s) 0.0651 | Train Loss 0.0713 | Val Loss 1.2613 | Train Accuracy 0.9889 | Val Accuracy 0.7306 
Epoch 00154 | Time(s) 0.0652 | Train Loss 0.0699 | Val Loss 1.2661 | Train Accuracy 0.9889 | Val Accuracy 0.7306 
Epoch 00155 | Time(s) 0.0651 | Train Loss 0.0684 | Val Loss 1.2709 | Train Accuracy 0.9889 | Val Accuracy 0.7306 
Epoch 00156 | Time(s) 0.0651 | Train Loss 0.0670 | Val Loss 1.2757 | Train Accuracy 0.98

## Implement random perturbations on victim nodes

In [4]:
# ---Simulate Non-malicious random connections---
graph_rc, target_nodes_list, target_mask = \
    random_connections(graph, test_mask, SAMPLE_RATE, PERT_RATE, TARGET_CLASS)
feat_rc = graph_rc.ndata['feat']

# ---Initialize the node classifier---
print("Initialize the node classifier...")
# Setup the GPU if necessary
if GPU < 0:
    print("Using CPU!")
    cuda = False
else:
    print("Using GPU!")
    cuda = True
    torch.cuda.set_device(GPU)
    graph_rc = graph_rc.to('cuda')
    feat_rc = feat_rc.cuda()
    target_mask = target_mask.cuda()
# Create the trained model    
model = GNN(g=graph_rc,
            in_feats=feat_rc.shape[1],
            n_hidden=N_HIDDEN,
            n_classes=len(torch.unique(label)),
            n_layers=N_LAYERS,
            activation=F.relu,
            dropout=DROPOUT,
            model_name=model_name,
            aggregator_type=kwargs_dicts[model_name][0],
            n_filter=kwargs_dicts[model_name][1],
            n_heads=kwargs_dicts[model_name][2])
if cuda: # if GPU is available
    model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
# Path for saving the parameters
path = 'runs/{0}_{1}/'.format(data_name, model_name) + 'model_best.pth.tar'

# ---Evaluation the trained GNN on the perturbed target nodes---
print("Evaluation on the target nodes.")
evaluation(model, optimizer, path, graph_rc, feat_rc, label, target_mask, cuda)

# ---Output the perturbed graph and the predictions---
print("Save the perturbed graph and the target mask.")
if cuda:
    graph_rc, target_mask = graph_rc.cpu(), target_mask.cpu()
dump_pickle(dirs_attack+'G_rdm_C{0}_{1}_{2}.pkl' \
            .format(TARGET_CLASS, data_name, model_name), \
            [graph_rc, target_nodes_list, target_mask])
print("Generate predicted labels after perturbation.")
Y_pred, Y_pred_sm = prediction(model, optimizer, path, graph_rc, feat_rc)
if cuda:
    Y_pred, Y_pred_sm = Y_pred.cpu(), Y_pred_sm.cpu()
dump_pickle('../data/noisy_label/Y_preds_rdmPert.pkl', [Y_pred, Y_pred_sm])
print("Y_pred/Y_pred_sm.shape: ", Y_pred.shape, Y_pred_sm.shape)


time:  0.03683209419250488
Initialize the node classifier...
Using CPU!
Evaluation on the target nodes.
Best Testing Accuracy: 48.95%
Test F1 Score: 41.97%
The distributions of groundtruth classes: 
 [22 11 27 67 32 22  9]
The distributions of predicted classes: 
 [  6   1  11 160  10   0   2]
The normalized entropy: 9.24% (±19.48%)
Save the perturbed graph and the target mask.
Generate predicted labels after perturbation.
Y_pred/Y_pred_sm.shape:  torch.Size([2708]) torch.Size([2708, 7])


## Employ LInDT model to infer labels

In [5]:
# load the perturbed graphs and the corresponding predicted labels
graph_name = 'G_rdm_C{0}_{1}_{2}.pkl'\
              .format(TARGET_CLASS, data_name, model_name)
graph, _, target_mask = read_pickle(dirs_attack+graph_name)
feat = graph.ndata['feat']
Y_pred, Y_pred_sm = read_pickle('../data/noisy_label/Y_preds_rdmPert.pkl')
# Reload the clean predicted labels for inference
Y_cpred, Y_cpred_sm = read_pickle('../data/noisy_label/Y_preds.pkl')
# Convert tensor to numpy array
Y_gt, Y_cpred, Y_noisy, Y_pred, Y_pred_sm, Y_cpred_sm = \
    label.numpy(), Y_cpred.numpy(), Y_noisy.numpy(), Y_pred.numpy(), \
    Y_pred_sm.detach().numpy(), Y_cpred_sm.detach().numpy()

# ---Initialize the initial warm-up transition matrix---
print("Initialize the warm-up transition matrix...")
Y_pred_sm_train = Y_cpred_sm[train_mask]  # predicted probability table (num_samples, num_classes)
Y_noisy_train = Y_noisy[train_mask]  # noisy labels
NUM_CLASSES = len(set(label.numpy()))
print("NUM_CLASSES: ", NUM_CLASSES)
TM_warmup = gen_init_trans_matrix(Y_pred_sm_train, Y_noisy_train, NUM_CLASSES)
print("The shape of warm-up TM is: ", TM_warmup.shape)

# ---Setup the gpu if necessary---
if GPU < 0:
    print("Using CPU!")
    cuda = False
else:
    print("Using GPU!")
    cuda = True
    torch.cuda.set_device(GPU)
    graph = graph.to('cuda')
    feat = feat.cuda()

# ---Initialize the node classifier---
print("Initialize the node classifier...")
model = GNN(g=graph,
            in_feats=feat.shape[1],
            n_hidden=N_HIDDEN,
            n_classes=len(torch.unique(label)),
            n_layers=N_LAYERS,
            activation=F.relu,
            dropout=DROPOUT,
            model_name=model_name,
            aggregator_type=kwargs_dicts[model_name][0],
            n_filter=kwargs_dicts[model_name][1],
            n_heads=kwargs_dicts[model_name][2])
if cuda:  # if gpu is available
    model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
# Path for saving the parameters
dirs = 'runs/{0}_{1}/'.format(data_name, model_name)
path = dirs + 'model_best.pth.tar'

# --- Label Inference jointly using Dirichlet and Topological sampling ---
if is_inferable:
    print("Infer the label...")
    Y_ssup_dict = {"pseudo": Y_cpred, "noise": Y_noisy}
    timer_0 = time.time()
    ss = LInDT(ALPHA)  # Y_noisy or Y_cpred
    Y_infer, C_new, _ = \
        ss.infer(model, optimizer, dirs, graph, feat, train_mask, val_mask, \
                 Y_pred, Y_pred_sm, Y_ssup_dict[infer_type], Y_gt, TM_warmup, \
                 NUM_RETRAIN, GPU, NUM_EPS, WARMUP_STEP, \
                 is_alert, is_dynamic_phi, Kernel_Type, is_dynamic_alpha)
    runtime = time.time() - timer_0
    print("\n Runtime: ", runtime)
    print("Y_inferred: \n {0} \n C_new: \n {1}".format(Y_infer, C_new))
print("Evaluation after inference:")
Y_infer, C_new, _ = read_pickle('../data/noisy_label/Y_C_RUN.pkl')  # array
_, cat_dist = prediction(model, optimizer, path, graph, feat)  # tensor
# Evaluation on test/target graph
mask_dict = {"test": test_mask, "target": target_mask}
assert len(mask_dict) > 0
for mask_name, mask in mask_dict.items():
    Y_gt_mask = label[mask].numpy()
    Y_infer_mask = Y_infer[mask]
    cat_dist_mask = cat_dist[mask].cpu() if cuda else cat_dist[mask]
    cat_dist_mask = cat_dist_mask.detach().numpy()
    acc_infer = accuracy_score(Y_gt_mask, Y_infer_mask)
    print("Accuracy of Y_infer on {} nodes: {:.2%}.".format(mask_name, acc_infer))
    f1_macro = f1_score(Y_gt_mask, Y_infer_mask, average='macro')
    f1_weighted = f1_score(Y_gt_mask, Y_infer_mask, average='weighted')
    print("F1 score (macro) of Y_infer on {} nodes: {:.2%}.".format(mask_name, f1_macro))
    print("F1 score (weighted) of Y_infer on {} nodes: {:.2%}.".format(mask_name, f1_weighted))
    total_entropy = entropy(cat_dist_mask, axis=1)
    normalized_entropy = total_entropy / np.log(cat_dist.shape[1])
    print("The normalized entropy on {} nodes: {:.2%}."\
          .format(mask_name, np.mean(normalized_entropy)))
    print("-"*30)


Initialize the warm-up transition matrix...
NUM_CLASSES:  7
The shape of warm-up TM is:  (7, 7)
Using CPU!
Initialize the node classifier...
Infer the label...
Update the model in step 0:


/Users/junzhuang/opt/miniconda3/envs/py39/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Epoch 00105 | Time(s) 0.0773 | Train Loss 1.6649 | Val Loss 2.5851 | Train Accuracy 0.8296 | Val Accuracy 0.7731 
Epoch 00106 | Time(s) 0.0737 | Train Loss 0.8789 | Val Loss 1.4896 | Train Accuracy 0.8778 | Val Accuracy 0.8100 
Epoch 00107 | Time(s) 0.0727 | Train Loss 0.3928 | Val Loss 0.7559 | Train Accuracy 0.9185 | Val Accuracy 0.8339 
Epoch 00108 | Time(s) 0.0730 | Train Loss 1.0155 | Val Loss 1.4238 | Train Accuracy 0.8148 | Val Accuracy 0.7011 
Epoch 00109 | Time(s) 0.0724 | Train Loss 0.9335 | Val Loss 1.2580 | Train Accuracy 0.8407 | Val Accuracy 0.7306 
Epoch 00110 | Time(s) 0.0717 | Train Loss 0.5297 | Val Loss 0.8003 | Train Accuracy 0.8963 | Val Accuracy 0.8118 
Epoch 00111 | Time(s) 0.0715 | Train Loss 0.3592 | Val Loss 0.5828 | Train Accuracy 0.9296 | Val Accuracy 0.8653 
Epoch 00112 | Time(s) 0.0713 | Train Loss 0.4839 | Val Loss 0.8239 | Train Accuracy 0.9111 | Val Accuracy 0.8524 
Epoch 00113 | Time(s) 0.0708 | Train Loss 0.5917 | Val Loss 1.0375 | Train Accuracy 0.88

/Users/junzhuang/opt/miniconda3/envs/py39/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Epoch 00156 | Time(s) 0.0704 | Train Loss 0.1645 | Val Loss 0.3166 | Train Accuracy 0.9815 | Val Accuracy 0.9520 
Epoch 00157 | Time(s) 0.0699 | Train Loss 0.1564 | Val Loss 0.2966 | Train Accuracy 0.9926 | Val Accuracy 0.9594 
Epoch 00158 | Time(s) 0.0687 | Train Loss 0.1563 | Val Loss 0.2889 | Train Accuracy 0.9963 | Val Accuracy 0.9557 
Epoch 00159 | Time(s) 0.0693 | Train Loss 0.1600 | Val Loss 0.2891 | Train Accuracy 0.9778 | Val Accuracy 0.9594 
Epoch 00160 | Time(s) 0.0685 | Train Loss 0.1606 | Val Loss 0.2901 | Train Accuracy 0.9704 | Val Accuracy 0.9631 
Epoch 00161 | Time(s) 0.0680 | Train Loss 0.1567 | Val Loss 0.2906 | Train Accuracy 0.9704 | Val Accuracy 0.9557 
Epoch 00162 | Time(s) 0.0675 | Train Loss 0.1521 | Val Loss 0.2934 | Train Accuracy 0.9778 | Val Accuracy 0.9483 
Epoch 00163 | Time(s) 0.0670 | Train Loss 0.1468 | Val Loss 0.2968 | Train Accuracy 0.9852 | Val Accuracy 0.9465 
Epoch 00164 | Time(s) 0.0665 | Train Loss 0.1422 | Val Loss 0.2992 | Train Accuracy 0.99

...

Epoch 00201 | Time(s) 0.0692 | Train Loss 0.0983 | Val Loss 0.2556 | Train Accuracy 1.0000 | Val Accuracy 0.9576 
Epoch 00202 | Time(s) 0.0689 | Train Loss 0.0975 | Val Loss 0.2540 | Train Accuracy 1.0000 | Val Accuracy 0.9576 
Epoch 00203 | Time(s) 0.0686 | Train Loss 0.0967 | Val Loss 0.2522 | Train Accuracy 1.0000 | Val Accuracy 0.9576 
Epoch 00204 | Time(s) 0.0685 | Train Loss 0.0959 | Val Loss 0.2508 | Train Accuracy 1.0000 | Val Accuracy 0.9539 
Epoch 00205 | Time(s) 0.0683 | Train Loss 0.0952 | Val Loss 0.2500 | Train Accuracy 1.0000 | Val Accuracy 0.9576 
Epoch 00206 | Time(s) 0.0682 | Train Loss 0.0945 | Val Loss 0.2494 | Train Accuracy 1.0000 | Val Accuracy 0.9576 
Epoch 00207 | Time(s) 0.0682 | Train Loss 0.0937 | Val Loss 0.2490 | Train Accuracy 1.0000 | Val Accuracy 0.9557 
Epoch 00208 | Time(s) 0.0682 | Train Loss 0.0929 | Val Loss 0.2488 | Train Accuracy 1.0000 | Val Accuracy 0.9557 
Epoch 00209 | Time(s) 0.0681 | Train Loss 0.0922 | Val Loss 0.2489 | Train Accuracy 1.00

/Users/junzhuang/opt/miniconda3/envs/py39/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Epoch 00202 | Time(s) 0.0700 | Train Loss 0.0975 | Val Loss 0.2540 | Train Accuracy 1.0000 | Val Accuracy 0.9576 
Epoch 00203 | Time(s) 0.0698 | Train Loss 0.0967 | Val Loss 0.2522 | Train Accuracy 1.0000 | Val Accuracy 0.9576 
Epoch 00204 | Time(s) 0.0693 | Train Loss 0.0959 | Val Loss 0.2508 | Train Accuracy 1.0000 | Val Accuracy 0.9539 
Epoch 00205 | Time(s) 0.0693 | Train Loss 0.0952 | Val Loss 0.2500 | Train Accuracy 1.0000 | Val Accuracy 0.9576 
Epoch 00206 | Time(s) 0.0691 | Train Loss 0.0945 | Val Loss 0.2494 | Train Accuracy 1.0000 | Val Accuracy 0.9576 
Epoch 00207 | Time(s) 0.0694 | Train Loss 0.0937 | Val Loss 0.2490 | Train Accuracy 1.0000 | Val Accuracy 0.9557 
Epoch 00208 | Time(s) 0.0693 | Train Loss 0.0929 | Val Loss 0.2488 | Train Accuracy 1.0000 | Val Accuracy 0.9557 
Epoch 00209 | Time(s) 0.0692 | Train Loss 0.0922 | Val Loss 0.2489 | Train Accuracy 1.0000 | Val Accuracy 0.9557 
Epoch 00210 | Time(s) 0.0690 | Train Loss 0.0916 | Val Loss 0.2488 | Train Accuracy 1.00

/Users/junzhuang/opt/miniconda3/envs/py39/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Epoch 00203 | Time(s) 0.0692 | Train Loss 0.0967 | Val Loss 0.2522 | Train Accuracy 1.0000 | Val Accuracy 0.9576 
Epoch 00204 | Time(s) 0.0692 | Train Loss 0.0959 | Val Loss 0.2508 | Train Accuracy 1.0000 | Val Accuracy 0.9539 
Epoch 00205 | Time(s) 0.0689 | Train Loss 0.0952 | Val Loss 0.2500 | Train Accuracy 1.0000 | Val Accuracy 0.9576 
Epoch 00206 | Time(s) 0.0687 | Train Loss 0.0945 | Val Loss 0.2494 | Train Accuracy 1.0000 | Val Accuracy 0.9576 
Epoch 00207 | Time(s) 0.0685 | Train Loss 0.0937 | Val Loss 0.2490 | Train Accuracy 1.0000 | Val Accuracy 0.9557 
Epoch 00208 | Time(s) 0.0685 | Train Loss 0.0929 | Val Loss 0.2488 | Train Accuracy 1.0000 | Val Accuracy 0.9557 
Epoch 00209 | Time(s) 0.0683 | Train Loss 0.0922 | Val Loss 0.2489 | Train Accuracy 1.0000 | Val Accuracy 0.9557 
Epoch 00210 | Time(s) 0.0683 | Train Loss 0.0916 | Val Loss 0.2488 | Train Accuracy 1.0000 | Val Accuracy 0.9557 
Epoch 00211 | Time(s) 0.0682 | Train Loss 0.0909 | Val Loss 0.2481 | Train Accuracy 1.00